In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo
import cx_Oracle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
# df_sgm = pd.read_csv('df_trans_user_item_date.csv', sep='\t', dtype='object')
# df_item = pd.read_csv('MASTER_ITEM_202012121853.csv', dtype='object')
# df_sgm['TR_DATE'] = df_sgm['TR_DATE'].map(lambda x: eval(x))
# df_sgm['TR_PLU'] = df_sgm['TR_PLU'].apply(lambda x: x[0:-1])

In [3]:
conn_str = u'report/justd0it@10.234.152.61:1521/alfabi'
connection = cx_Oracle.connect(conn_str)

conn_mongo = pymongo.MongoClient\
        ("mongodb://127.0.0.1:27017/seq_trans")
mydb = conn_mongo["seq_trans"]
print(mydb.list_collection_names())

mycol = mydb['member_item_seq']

['member_item_seq']


In [4]:
# q_plu = """
# SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
# where lower(mpa.MPA_DESC) like 'sgm eksplor%'
# """

# df_plu = pd.read_sql_query(q_plu, connection)     
# li_plu = df_plu['MPA_PLU']


# df_all = []
# for plu in li_plu[0:20]:
#     df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
#     df_all.append(df)

# df_seq_1 = pd.concat(df_all)
# df_seq_1.head()
df_seq_1 = pd.DataFrame()

In [ ]:
q_plu = """
SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
where lower(mpa.MPA_DESC) like 'sgm eksplor%'
or lower(mpa.MPA_DESC) like 'anlene%'
or lower(mpa.MPA_DESC) like 'dancow%'
or lower(mpa.MPA_DESC) like 'bebelac%'
or lower(mpa.MPA_DESC) like 'lactogrow%'
or lower(mpa.MPA_DESC) like 'ff baby%'
or lower(mpa.MPA_DESC) like 'pepsodent pg economy%'
"""

df_plu = pd.read_sql_query(q_plu, connection)     
li_plu = df_plu['MPA_PLU']


df_all = []
for plu in li_plu:
    df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
    df_all.append(df)

df_seq_2 = pd.concat(df_all)
df_seq_2.head()

In [ ]:
df_seq = pd.concat([df_seq_1, df_seq_2]).reset_index(drop=True)

In [ ]:
q_nas = """
SELECT ITEM_N, DESCP, UKURAN, PANJANG, LEBAR, TINGGI, UOM FROM MASTER_NAS_M mnm 
"""

df_nas = pd.read_sql_query(q_nas, connection)
df_nas.head()

In [ ]:
df = pd.merge(df_seq, df_nas, left_on='TR_PLU_PCS', right_on='ITEM_N')[list(df_seq) + ['DESCP','UKURAN']]
df = df[df['TRX_DATE'].notnull()]

min_freq = 6
df_hist = df[df['TRX_DATE'].apply(lambda x: len(x)) >= min_freq]

In [ ]:
df_hist.head()

In [ ]:
# set_time = {'1608163200': 1, '1607904000': 1, '1607040000': 1, '1606176000': 1, '1605916800': 1, '1605657600': 1, '1605225600': 1, '1602288000': 1}
# set_time = sorted(set_time.items())


# print(set_time)

In [ ]:
def cal_feature(set_timestamp):

    set_timestamp = sorted(set_timestamp.items())
    last_order = int(set_timestamp[-1][0])
    
    li_seq = []
    qty_seq = []
    for idx in range(0, len(set_timestamp)-1, 1):

        diff = int(set_timestamp[idx+1][0]) - int(set_timestamp[idx][0])
        diff_day = int((diff / 3600) / 24)
        li_seq.append(diff_day)
        
    qty_seq = [i[1] for i in set_timestamp][-1*(min_freq):-1]

    li_seq = li_seq[-1*(min_freq-1):]
    # max diff day
    seq_max = np.max(li_seq)
    # min diff day
    seq_min = np.min(li_seq)
    # day since
    seq_since = li_seq[-2]
    # mean
    seq_mean = round(np.mean(li_seq),2)
    # mean_qty
    seq_mean_qty = round(np.mean([li_seq[i]/qty_seq[i] for i in range(len(li_seq))]),2)
    # variance
    seq_var = round(np.var(li_seq),2)

    ## appending
    li_seq.append(seq_max)
    li_seq.append(seq_min)
    li_seq.append(seq_since)
    li_seq.append(seq_mean)
    li_seq.append(seq_var)
    
    li_seq.extend(qty_seq)
    li_seq.append(seq_mean_qty)


    return li_seq, last_order

In [ ]:
df_hist['tr_date_seq'], df_hist['last_ord1er'] = zip(*df_hist['TRX_DATE'].apply(cal_feature))

In [ ]:
#col_feature = ['t_5', 't_4', 't_3', 't_2', 't_1', 'max', 'min', 'since', 'mean', 'variance', 'q_5', 'q_4', 'q_3', 'q_2', 'q_1', 'mean_qty']
col_feature = ['t_'+ str(i) for i in range((min_freq-1),0,-1)] + \
                ['max', 'min', 'since', 'mean', 'variance'] + \
                ['q_'+ str(i) for i in range((min_freq-1),0,-1)] + ['mean_qty']
train = pd.DataFrame(df_hist.tr_date_seq.tolist(), index=df_hist.index, columns=col_feature)

# X = train.drop('t_0', 1)
# Y = train['t_0']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
train.head()

### Compare time consuming in different netto
> 
* first we user with less variance, we assume user buy based on their consumption and less affected by promotion or random purchase date
* age might be affect consumption, cause more mature, amount of consumption should be decreased by parent

In [ ]:
# with open('watchlist_item.txt') as f:
#     watchlist = f.read().splitlines()
#     df_watchlist = pd.DataFrame({'DESCP':watchlist})

In [ ]:
train_descp = pd.concat([train,df_hist], 1)[col_feature + ['TR_MEMBERS', 'TR_PLU_PCS', 'DESCP', 'TRX_DATE', 'UKURAN']]
train_descp = pd.merge(train_descp, df_plu[['MPA_PLU', 'MPA_DESC_SUBCAT']], left_on='TR_PLU_PCS', right_on='MPA_PLU', how='left')
train_descp = train_descp[train_descp['UKURAN'].notnull()]
train_descp['UKURAN'] = train_descp['UKURAN'].astype(int)

In [ ]:
# train_descp.groupby('UKURAN').agg({'mean_qty':'mean'})

# train_descp_cond = train_descp[train_descp['variance'] < 100]
# train_descp_cond.groupby('UKURAN').agg({'mean_qty':'mean'})

In [ ]:
train_descp['DESCP_TYPE'] = train_descp['DESCP'].str.split(' ').str[0:-3].str.join(' ')

In [ ]:
train_descp.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN']).agg({'mean_qty':'mean'})

In [ ]:
train_descp_cond = train_descp[train_descp['variance'] < 80]
train_descp_cond.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN']).agg({'mean_qty':'mean'})